In [1]:
!pip install mysql
!pip install mysql-connector-python
!pip install mysql-connector-python-rf
!pip install csv
!pip install ultralytics
!pip install easyocr
!pip install opencv-python
!pip install regex
!pip install collections
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mysqlclient: filename=mysqlclient-2.2.7-cp312-cp312-linux_x86_64.whl size=129066 sha256=5cf1b2867c432fe429730be19468a1fc242b671170c0cdd47061086c4bb26d29
  Stored in directory: /root/.cache/pip/wheels/27/95/18/7f176fffd46629e710c04c810b9c4d7d4358fe7c96a7d2306d
Successfully built mysqlclient
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 71.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql-connector-python-rf: filename=mysql_connector_python_rf-2.2.2-cp312-cp312-linux_x86_64.whl size=249460 sha256=041071665f90697769b2172fbf34b5164e8f1c1e0f7523f8fb5c76a9619e20d2
  Stored in directory: /root/.cache/pip/wheels/a9/2b/d7/d900eb4840457595

In [6]:
import argparse
import cv2
import easyocr
import numpy as np
import pandas as pd
import re
import time
import torch
from ultralytics import YOLO
from tqdm import tqdm
import os

# --- Configuration Constants ---
# Simple regex for Indian license plates (e.g., TN 01 AA 0001)
# Allows optional spaces.
LICENSE_PLATE_REGEX = r"^[A-Z]{2}[0-9]{1,2}[A-Z]{1,2}[0-9]{4}$"
# Time window for de-duplication (in seconds)
DEDUP_WINDOW_SEC = 3.0
# Detection confidence threshold
DETECTION_CONF_THRESHOLD = 0.3
# Margin in pixels to expand the cropped license plate image
CROP_MARGIN = 10
# Target size for the plate crop before OCR (improves OCR accuracy)
OCR_TARGET_SIZE = (320, 100)


def load_models(model_path, device):
    """Loads the YOLOv8 model and EasyOCR reader."""
    print(f"Loading YOLOv8 model from {model_path}...")
    yolo_model = YOLO(model_path)

    print(f"Loading EasyOCR reader on device: {device}...")
    # EasyOCR only supports 'cuda' or 'cpu', matching torch's device logic
    ocr_device = 'cuda' if 'cuda' in device else 'cpu'
    ocr_reader = easyocr.Reader(['en'], gpu=('cuda' in ocr_device))

    return yolo_model, ocr_reader

def postprocess_text(text):
    """
    Cleans OCR text: upper-case, trim whitespace, remove non-alphanumeric except hyphen.
    """
    # 1. Convert to upper case and trim whitespace
    cleaned_text = text.strip().upper()

    # 2. Remove all non-alphanumeric characters except space
    # The space characters are needed for the regex validation below,
    # but we'll remove them for the final standardized output.

    # 3. Standardize by removing all spaces and hyphens first to check against a strict regex
    standardized_text = re.sub(r'[^A-Z0-9]', '', cleaned_text)

    return standardized_text, cleaned_text

def run_ocr_on_crop(reader, crop_img):
    """
    Applies preprocessing and runs EasyOCR on the cropped image.
    """
    # 1. Preprocessing: Grayscale and Resize
    if crop_img.shape[0] > 0 and crop_img.shape[1] > 0:
        gray_crop = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        resized_crop = cv2.resize(gray_crop, OCR_TARGET_SIZE, interpolation=cv2.INTER_CUBIC)
    else:
        return "", 0.0

    # Optional: Simple thresholding (can sometimes help for very dark/bright plates)
    # _, thresh_crop = cv2.threshold(resized_crop, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 2. Run EasyOCR
    # reader.readtext returns a list of (bbox, text, confidence)
    ocr_results = reader.readtext(resized_crop, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')

    plate_text = ""
    avg_conf = 0.0

    if ocr_results:
        # EasyOCR can sometimes split text. Concatenate the text parts.
        all_texts = [res[1] for res in ocr_results]
        all_confs = [res[2] for res in ocr_results]

        plate_text = " ".join(all_texts)
        avg_conf = np.mean(all_confs) if all_confs else 0.0

    return plate_text, avg_conf

def process_video(yolo_model, ocr_reader, video_path, output_video_path, output_csv_path):
    """Main function to process the video frame by frame."""

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Error: Cannot open video file {video_path}")

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Use 'mp4v' for MP4 output

    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    results_list = []
    # De-duplication tracking: stores {plate_text: timestamp_s}
    seen_plates = {}

    print(f"Processing video: {video_path} ({total_frames} frames @ {fps:.2f} FPS)")

    # Use tqdm for progress tracking
    for frame_no in tqdm(range(total_frames), desc="Processing Frames"):
        ret, frame = cap.read()
        if not ret:
            break

        timestamp_s = frame_no / fps

        # 1. YOLOv8 Detection
        yolo_results = yolo_model.predict(source=frame, verbose=False, conf=DETECTION_CONF_THRESHOLD)

        annotated_frame = frame.copy()

        if yolo_results and len(yolo_results[0].boxes) > 0:
            for box in yolo_results[0].boxes:
                # Get detection info
                conf = float(box.conf[0].cpu().numpy())

                # Get Bounding Box coordinates (x1, y1, x2, y2)
                x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())

                # Apply margin for cropping (ensure coordinates stay within frame bounds)
                x1_crop = max(0, x1 - CROP_MARGIN)
                y1_crop = max(0, y1 - CROP_MARGIN)
                x2_crop = min(frame_width, x2 + CROP_MARGIN)
                y2_crop = min(frame_height, y2 + CROP_MARGIN)

                # Crop the license plate region
                plate_crop = frame[y1_crop:y2_crop, x1_crop:x2_crop]

                # 2. EasyOCR Recognition
                raw_plate_text, ocr_conf = run_ocr_on_crop(ocr_reader, plate_crop)

                # 3. Post-processing and Validation
                standardized_text, display_text = postprocess_text(raw_plate_text)

                is_validated = bool(re.match(LICENSE_PLATE_REGEX, standardized_text))

                # 4. De-duplication check
                is_duplicate = False
                if standardized_text and standardized_text in seen_plates:
                    last_seen_time = seen_plates[standardized_text]
                    if (timestamp_s - last_seen_time) < DEDUP_WINDOW_SEC:
                        is_duplicate = True
                    else:
                        # Plate seen after dedup window, update timestamp
                        seen_plates[standardized_text] = timestamp_s
                elif standardized_text:
                    # New plate seen
                    seen_plates[standardized_text] = timestamp_s

                # 5. Record Result
                if not is_duplicate and standardized_text:
                    # Console print
                    print(f"| F{frame_no:5d} | T{timestamp_s:.2f}s | BBOX({x1},{y1},{x2},{y2}) | Plate: {standardized_text} | Conf: {ocr_conf:.2f} | Validated: {is_validated}")

                    # CSV Record
                    results_list.append({
                        'frame_no': frame_no,
                        'timestamp_s': timestamp_s,
                        'camera_id': 'CAM01', # Placeholder
                        'plate_text': standardized_text,
                        'ocr_confidence': ocr_conf,
                        'detection_confidence': conf,
                        'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2,
                        'validated': is_validated
                    })

                # 6. Annotation for Output Video

                # Annotation color: Red for detected box
                color = (0, 0, 255)
                # Draw bounding box
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)

                # Annotation text: display_text (allows spaces/hyphens for readability)
                text_label = f"PLATE: {display_text}" if display_text else "..."

                # Put text above the bounding box
                cv2.putText(annotated_frame, text_label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Write the annotated frame to the output video
        out.write(annotated_frame)

    # Clean up
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Save results to CSV
    if results_list:
        df = pd.DataFrame(results_list)
        df.to_csv(output_csv_path, index=False)
        print(f"\n✅ Results saved to CSV: {output_csv_path}")
    else:
        print("\n⚠️ No license plates were successfully recognized and recorded.")

    print(f"✅ Annotated video saved to: {output_video_path}")


if __name__ == '__main__':

    # --- Check for torch and device ---
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

    # --- Define paths directly ---
    # TODO: Update these paths to your actual model and video files
    model_path = '/content/best.pt'  # Example: Path to your YOLOv8 model
    input_video_path = '/content/test_video.mp4'  # Example: Path to your input video
    output_video_path = '/content/output_video.mp4'
    output_csv_path = '/content/anpr_results.csv'

    # --- Run Process ---
    try:
        yolo_model, ocr_reader = load_models(model_path, DEVICE)
        # Ensure YOLO uses the specified device
        yolo_model.to(DEVICE)

        process_video(yolo_model, ocr_reader, input_video_path, output_video_path, output_csv_path)

    except Exception as e:
        print(f"\n--- FATAL ERROR --- \nAn error occurred during execution: {e}")

Loading YOLOv8 model from /content/best.pt...

--- FATAL ERROR --- 
An error occurred during execution: [Errno 2] No such file or directory: '/content/best.pt'


In [5]:
# --- Define paths for your files ---
# IMPORTANT: Update these paths to your actual model and video files

# Path to your YOLOv8 model file (e.g., from your Google Drive)
model_path = '/content/drive/MyDrive/capstone project/model Weight/license_plate_best.pt'

# Path to your input video file
input_video_path = '/content/drive/MyDrive/capstone project/videos/numberplate test.mp4'

# Output path for the annotated video (this will be created)
output_video_path = '/content/output_video.mp4'

# Output path for the results CSV file (this will be created)
output_csv_path = '/content/anpr_results.csv'

print(f"Model path set to: {model_path}")
print(f"Input video path set to: {input_video_path}")
print(f"Output video path set to: {output_video_path}")
print(f"Output CSV path set to: {output_csv_path}")


Model path set to: /content/drive/MyDrive/capstone project/model Weight/license_plate_best.pt
Input video path set to: /content/drive/MyDrive/capstone project/videos/numberplate test.mp4
Output video path set to: /content/output_video.mp4
Output CSV path set to: /content/anpr_results.csv


In [7]:
# --- Run the video processing ---
# This code will use the paths defined in the previous cell.

# --- Check for torch and device ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

try:
    # Load models using the specified paths
    yolo_model, ocr_reader = load_models(model_path, DEVICE)
    # Ensure YOLO uses the specified device
    yolo_model.to(DEVICE)

    # Process the video
    process_video(yolo_model, ocr_reader, input_video_path, output_video_path, output_csv_path)

except Exception as e:
    print(f"\n--- FATAL ERROR --- \nAn error occurred during execution: {e}")


Loading YOLOv8 model from /content/best.pt...

--- FATAL ERROR --- 
An error occurred during execution: [Errno 2] No such file or directory: '/content/best.pt'


In [8]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed